In [36]:
import sportsdataverse.nfl as nfl 
from bs4 import BeautifulSoup
import os
#import urllib.request
import requests
import pandas as pd
import numpy as np
import scipy
import difflib
import json
#import matplotlib.pyplot as plt
import time
import scipy
import statistics as stats
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import nfl_data_py as nfl_dp

week = 11


In [37]:
# --------------------------- load and prepare data -------------------- #
pbp = nfl.load_nfl_pbp(seasons=([2024]),return_as_pandas=True)
data=pbp.copy()

pbp = pbp[((pbp['pass'] == 1) | (pbp['rush'] == 1)) & ( pbp.season_type == "REG")]
mask = {'home':1,'away':0}
pbp.posteam_type = pbp.posteam_type.map(mask)


cols = pbp.select_dtypes(include = 'number').columns
df_off = pbp.groupby(['posteam','game_id'])[cols].agg(['min','max','sum','median','mean','size'])
df_def = pbp.groupby(['defteam','game_id'])[cols].agg(['min','max','sum','median','mean','size'])


df_off = pbp.groupby(['posteam'])[cols].agg(['min','max','sum','median','mean','size'])
df_def = pbp.groupby(['defteam'])[cols].agg(['min','max','sum','median','mean','size'])

df_off.index = np.array(df_off.index) + '_off'
df_def.index = np.array(df_def.index) + '_def'

df = pd.concat([df_off,df_def])
df[[('pass_ypa','value')]] =  df[[('passing_yards','sum')]].values/df[[('pass_attempt','sum')]].values
df[[('rush_ypa','value')]] =  df[[('rushing_yards','sum')]].values/df[[('rush_attempt','sum')]].values
df[[('success_rate','value')]] =  df[[('success','sum')]].values/df[[('success','size')]].values

df = df.sort_index()

df_off = df.loc[df.index.str.contains('_off')]
df_def = df.loc[df.index.str.contains('_def')]



100%|██████████| 1/1 [00:00<00:00,  2.18it/s]


In [34]:
pbp.columns[350:450]

Index(['special', 'play', 'passer_id', 'rusher_id', 'receiver_id', 'name',
       'jersey_number', 'id', 'fantasy_player_name', 'fantasy_player_id',
       'fantasy', 'fantasy_id', 'out_of_bounds', 'home_opening_kickoff',
       'qb_epa', 'xyac_epa', 'xyac_mean_yardage', 'xyac_median_yardage',
       'xyac_success', 'xyac_fd', 'xpass', 'pass_oe'],
      dtype='object')

In [38]:
# --------------------------- offense rank quick view -------------------- #
offense = df_off[[('epa','mean'),('passing_yards','sum'),('pass_ypa','value'),('rush_ypa','value'),('success_rate','value'),('pass_oe','mean')]].rank(ascending=False).astype('int')
df_off[[('epa','mean'),('passing_yards','sum'),('pass_ypa','value'),('rush_ypa','value'),('success_rate','value'),('pass_oe','mean')]].rank(ascending=False).astype('int').style.background_gradient(cmap='coolwarm')


,epa,passing_yards,pass_ypa,rush_ypa,success_rate,pass_oe
,mean,sum,value,value,value,mean
ARI_off,8,21,11,2,6,24
ATL_off,10,3,7,9,4,23
BAL_off,1,2,1,1,1,27
BUF_off,3,10,9,13,7,9
CAR_off,30,29,29,12,28,21
CHI_off,25,30,30,24,27,13
CIN_off,5,1,13,25,11,1
CLE_off,31,26,32,29,32,3
DAL_off,28,13,24,30,24,11


In [16]:
# --------------------------- defense rank quick view -------------------- #
defense = df_def[[('epa','mean'),('passing_yards','sum'),('pass_ypa','value'),('rush_ypa','value'),('success_rate','value')]].rank(ascending=True).astype('int')
df_def[[('epa','mean'),('passing_yards','sum'),('pass_ypa','value'),('rush_ypa','value'),('success_rate','value')]].rank(ascending=True).astype('int').style.background_gradient(cmap='coolwarm')


,epa,passing_yards,pass_ypa,rush_ypa,success_rate
,mean,sum,value,value,value
ARI_def,29,22,30,20,32
ATL_def,22,10,9,13,26
BAL_def,23,32,28,1,9
BUF_def,15,21,8,30,8
CAR_def,30,23,31,18,29
CHI_def,3,11,6,21,4
CIN_def,26,15,15,16,31
CLE_def,19,13,20,26,11
DAL_def,27,18,27,19,17


In [17]:
import nfl_data_py as nfl_dp
s = nfl_dp.import_schedules(years=[2024])


In [20]:
s = s[s['week'] == week]

In [21]:
away_team = 'ATL'
home_team = 'SEA'


In [22]:
ho = offense.loc[home_team+'_off',:]
ad = defense.loc[away_team+'_def',:]

hd = defense.loc[home_team+'_def',:]
ao = offense.loc[away_team+'_off',:]

In [29]:
pd.DataFrame([ho,ad]).fillna(0).astype('int').style.background_gradient(cmap='coolwarm')

,epa,passing_yards,pass_ypa,rush_ypa,success_rate,pass_oe
,mean,sum,value,value,value,mean
SEA_off,16,1,16,11,9,1
ATL_def,22,10,9,13,26,0


In [32]:
pd.DataFrame([hd,ao]).fillna(0).astype('int').style.background_gradient(cmap='coolwarm')

,epa,passing_yards,pass_ypa,rush_ypa,success_rate,pass_oe
,mean,sum,value,value,value,mean
SEA_def,17,14,17,27,15,0
ATL_off,8,5,7,10,11,17
